In [1]:
import torch

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys
sys.path.append('../')

In [6]:
import pytorch_lightning as pl
import lattice
from trainer import Trainer

### Make hparams from default_hparams

In [7]:
hparams = lattice.Q_DEFAULT_HPARAMS

In [8]:
hparams.potential = 'ising_potential'
hparams.passive_rates = 'ising_passive_rates'
# hparams.J = 0.32424925229  # Hamer pseudo-critical point for 4x4 # E0 goal: −1.066430308096
# hparams.lattice_size = 4
hparams.J = 0.32758326752  # Hamer pseudo-critical point for 6x6 # E0 goal: -1.063752757694
hparams.lattice_size = 6
hparams.g = 1.0
hparams.E = -1.0

# hparams.J = 1.0
# hparams.lattice_size = 4
# hparams.potential = 'xy_potential'
# hparams.passive_rates = 'xy_passive_rates'
# hparams.E = -1.0

# hparams.potential = 'heisenberg_antiferromagnet_potential'
# hparams.passive_rates = 'heisenberg_antiferromagnet_passive_rates'
# hparams.J = 1.0
# hparams.lattice_size = 4
# hparams.E = 0.5
# hparams.validate_fn = 'local_stoquastic_energy'

# hparams.potential = 'bh_potential'
# hparams.passive_rates = 'bh_passive_rates'
# hparams.E = -1.0
# hparams.lattice_size = 4

In [9]:
hparams.concentration = 0.5001

hparams.lr = 5e-4
hparams.sync_rate = 20
hparams.batch_size = 1024
hparams.buffer_capacity = 64 * hparams.batch_size
hparams.tau = 1e-6

hparams.gamma_lr = 1.0
hparams.hidden_channels = 32
hparams.num_steps = 64
hparams.epoch_size = 20

In [10]:
print(hparams)

Namespace(E=-1.0, J=0.32758326752, batch_size=1024, buffer_capacity=65536, concentration=0.5001, epoch_size=20, g=1.0, gamma_lr=1.0, git_commit='27c8f5c', hidden_channels=32, holding=True, kernel_size=3, lattice_size=4, loss_fn='bellman', lr=0.0005, net='QPeriodicCNN', num_steps=64, passive_rates='ising_passive_rates', potential='ising_potential', sync_rate=20, tau=1e-06, validate_fn='local_stoquastic_energy')


## Define model

In [11]:
model = lattice.QModel(hparams)

## Train model

In [12]:
trainer = Trainer(name='ising', max_epochs=100, nb_sanity_val_steps=1)
# checkpoint_path = 'results/j1j2/version_61/_ckpt_epoch_49.ckpt'
# # model = lattice.CModel.load_from_checkpoint(checkpoint_path)
# # # model.load_state_dict(model2.state_dict())
# # # model = model.cuda()
# # model.eval()
# trainer = Trainer(name='j1j2', gpus=[1], max_epochs=100, nb_sanity_val_steps=0,
#                   version=61, resume_from_checkpoint=checkpoint_path)


INFO:lightning:GPU available: False, used: False


In [12]:
# torch.autograd.set_detect_anomaly(True)

In [13]:
trainer.fit(model)

INFO:lightning:
   | Name                         | Type         | Params
----------------------------------------------------------
0  | Q_fn                         | QPeriodicCNN | 19 K  
1  | Q_fn.initial_layer           | Sequential   | 320   
2  | Q_fn.initial_layer.0         | Conv2d       | 320   
3  | Q_fn.initial_layer.1         | ReLU         | 0     
4  | Q_fn.inner_layers            | ModuleList   | 18 K  
5  | Q_fn.inner_layers.0          | Sequential   | 9 K   
6  | Q_fn.inner_layers.0.0        | Conv2d       | 9 K   
7  | Q_fn.inner_layers.0.1        | ReLU         | 0     
8  | Q_fn.inner_layers.1          | Sequential   | 9 K   
9  | Q_fn.inner_layers.1.0        | Conv2d       | 9 K   
10 | Q_fn.inner_layers.1.1        | ReLU         | 0     
11 | Q_fn.final_layer             | Conv2d       | 289   
12 | Q_target_fn                  | QPeriodicCNN | 19 K  
13 | Q_target_fn.initial_layer    | Sequential   | 320   
14 | Q_target_fn.initial_layer.0  | Conv2d       | 320 

1

In [14]:
print(model.minHss)

-0.2666666805744171


### Load trained model

In [16]:
# checkpoint_path = 'results/ising/version_5/_ckpt_epoch_1.ckpt'
# model = lattice.Model.load_from_checkpoint(checkpoint_path)

### Resume training

In [17]:
# trainer = Trainer(name='H2', gpus=[3],
#                   version=0, resume_from_checkpoint=checkpoint_path)
# trainer.fit(model)